In [1]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt
from ultralytics.engine.results import Results
from ultralytics import YOLO
import numpy as np
from PIL import Image, ExifTags
import torch

In [2]:
# INPUT_DIRS
INPUT_DATA_DIR = Path('dataset')
## Drop the Folder if it already exists
DATASETS_DIR = Path('dataset')
# Image & labels directory
TRAIN_IMAGES_DIR = DATASETS_DIR / 'images' / 'train'
TRAIN_LABELS_DIR = DATASETS_DIR / 'labels'/ 'train'
TEST_IMAGES_DIR = DATASETS_DIR / 'images' / 'test'
VAL_IMAGES_DIR = DATASETS_DIR / 'images' /'val'
VAL_LABELS_DIR = DATASETS_DIR / 'labels' /'val'

# Load train and test files
train = pd.read_csv(INPUT_DATA_DIR / 'Train_df.csv')
val = pd.read_csv(INPUT_DATA_DIR / 'Val_df.csv')
test = pd.read_csv(INPUT_DATA_DIR / 'Test.csv')
ss = pd.read_csv(INPUT_DATA_DIR / 'SampleSubmission.csv')

class_map = {cls: i for i, cls in enumerate(sorted(train['class'].unique().tolist()))}
# Strip any spacing from the class item and make sure that it is a str
train['class'] = train['class'].str.strip()

# Map {'healthy': 2, 'cssvd': 1, anthracnose: 0}
train['class_id'] = train['class'].map(class_map)

train_df = train
val_df = val

# Create a data.yaml file required by yolo
class_names = sorted(train['class'].unique().tolist())
num_classes = len(class_names)
data_yaml = {
    "path" : str(DATASETS_DIR.absolute()),
    'train': str(TRAIN_IMAGES_DIR.absolute()),
    'val': str(VAL_IMAGES_DIR.absolute()),
    'test': str(TEST_IMAGES_DIR.absolute()),
    'nc': num_classes,
    'names': class_names
}

val_image_names = [str(Path(name).stem) for name in val_df['Image_ID'].unique()]
train_image_names = [str(Path(name).stem) for name in train['ImagePath'].unique()]

In [3]:
from glob import glob

# Validate the model on the validation set
BEST_PATH = sorted(glob("runs/detect/train*/weights/best.pt"))[-1]
BEST_PATH = "runs/detect/train3/weights/best.pt"
BEST_PATH

'runs/detect/train3/weights/best.pt'

In [4]:
for flag, v in ExifTags.TAGS.items():
    if v == "Orientation":
        break


def load_image(filepath):
    image = Image.open(filepath)

    exif = image._getexif()
    if exif is None:
        return image

    orientation_value = exif.get(flag, None)

    if orientation_value == 3:
        image = image.rotate(180, expand=True)
    elif orientation_value == 6:
        image = image.rotate(270, expand=True)
    elif orientation_value == 8:
        image = image.rotate(90, expand=True)
    return image


flag

274

In [5]:
# Load the trained YOLO model
model = YOLO(BEST_PATH)

# Path to the test images directory
test_dir_path = TEST_IMAGES_DIR

# Get a list of all image files in the test directory
image_files = os.listdir(test_dir_path)

# Initialize an empty list to store the results for all images
all_data = []

# Initialize an empty list to store the results for all images
all_data = []

# Iterate through each image in the directory
for image_file in tqdm(image_files):
    # Full path to the image
    img_path = os.path.join(test_dir_path, image_file)

    # Make predictions on the image
    results: list[Results] = model(
        load_image(img_path),
        conf=0.0,
        imgsz=1024,
        max_det=30,
        verbose=False,
        # nms=True,
    )

    # Extract bounding boxes, confidence scores, and class labels
    boxes = (
        results[0].boxes.xyxy.tolist() if results[0].boxes else []
    )  # Bounding boxes in xyxy format
    classes = results[0].boxes.cls.tolist() if results[0].boxes else []  # Class indices
    confidences = (
        results[0].boxes.conf.tolist() if results[0].boxes else []
    )  # Confidence scores
    names = results[0].names  # Class names dictionary

    if boxes:  # If detections are found
        for box, cls, conf in zip(boxes, classes, confidences):
            x1, y1, x2, y2 = box
            detected_class = names[
                int(cls)
            ]  # Get the class name from the names dictionary

            # Add the result to the all_data list
            all_data.append(
                {
                    "Image_ID": str(image_file),
                    "class": detected_class,
                    "confidence": conf,
                    "ymin": y1,
                    "xmin": x1,
                    "ymax": y2,
                    "xmax": x2,
                }
            )
    else:  # If no objects are detected
        all_data.append(
            {
                "Image_ID": str(image_file),
                "class": None,
                "confidence": None,
                "ymin": None,
                "xmin": None,
                "ymax": None,
                "xmax": None,
            }
        )

100%|██████████| 1626/1626 [03:35<00:00,  7.53it/s]


In [6]:
# Convert the list to a DataFrame for all images
sub = pd.DataFrame(all_data)

In [7]:
sub.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,ID_cWEAQI.jpeg,healthy,0.587939,196.297638,10.367512,3934.849121,1636.239014
1,ID_cWEAQI.jpeg,healthy,0.041469,174.486389,0.000000,1687.742798,498.413879
2,ID_cWEAQI.jpeg,healthy,0.030204,349.489777,1338.696899,1136.139893,1795.408813
3,ID_cWEAQI.jpeg,healthy,0.020180,101.959702,1.075506,2287.603271,567.269043
4,ID_cWEAQI.jpeg,healthy,0.014658,0.958770,1253.013062,477.129944,1795.990356


In [8]:
sub.describe()

,confidence,ymin,xmin,ymax,xmax
count,48780.000000,48780.000000,48780.000000,48780.000000,48780.000000
mean,0.044775,601.289915,590.529700,1459.358144,1270.730227
std,0.140986,830.659779,785.794356,1147.139502,989.155295
min,0.000151,0.000000,0.000000,11.972401,16.580347
25%,0.001779,0.453291,0.567805,510.987267,474.298698
50%,0.004242,197.447174,244.482697,1102.946167,959.843231
75%,0.012456,921.049561,858.602966,2071.401001,1801.543732
max,0.882056,3933.594482,3918.724854,4128.000000,4128.000000


In [9]:
sub['class'].value_counts()

class
healthy        20672
cssvd          15481
anthracnose    12627
Name: count, dtype: int64

In [10]:
sub.isna().sum()

Image_ID      0
class         0
confidence    0
ymin          0
xmin          0
ymax          0
xmax          0
dtype: int64

class
healthy        1153
cssvd           801
anthracnose     694
None             57
Name: count, dtype: int6

In [11]:
sub.to_csv("dataset/predictions/02-predictions.csv", index=False)

In [12]:
sub["confidence"].describe()

count    48780.000000
mean         0.044775
std          0.140986
min          0.000151
25%          0.001779
50%          0.004242
75%          0.012456
max          0.882056
Name: confidence, dtype: float64

In [13]:
import pandas as pd

sub = pd.read_csv('dataset/predictions/02-predictions.csv')

sub.sample(6)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
33677,ID_xy2IQY.jpg,anthracnose,0.003350,0.000000,0.000000,593.430298,575.294617
24557,ID_ZwY4OM.jpg,healthy,0.004151,0.317456,0.000000,767.924744,492.427124
40849,ID_RDuHV3.jpg,cssvd,0.004660,4.808149,700.730286,856.552368,809.913330
14773,ID_rgDeQr.jpg,healthy,0.004727,0.159482,100.877838,504.040161,1173.864502
18323,ID_Kwu8Oa.jpg,healthy,0.001682,117.706451,182.313721,231.762497,285.270203
9464,ID_Z7ar8k.jpg,healthy,0.001420,31.147079,0.090029,272.535675,38.211124


In [14]:
sub["Image_ID"].value_counts().describe()

count    1626.0
mean       30.0
std         0.0
min        30.0
25%        30.0
50%        30.0
75%        30.0
max        30.0
Name: count, dtype: float64

In [15]:
sub["Image_ID"].nunique()

1626

In [16]:
sub.isna().sum()

Image_ID      0
class         0
confidence    0
ymin          0
xmin          0
ymax          0
xmax          0
dtype: int64